In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Use the citipy module to determine city based on latitude and longitude.
    # Under "Looking up with coordinates," the first line says from citipy import citipy, meaning we'll import the citipy script from the citipy module.
    # When a Python file containing a script is imported to use in another Python script, the .py extension does not need to be added to the name of the file when using the import statement.    
from citipy import citipy

# Import the datetime module from the datetime library.
    # If we want to convert the timestamp to the International Organization for Standardization (ISO) format, or YYYY-MM-DD-HH-MM-SS, we need to use the Python datetime module.
from datetime import datetime

# Import the API key.
    # When a Python file containing a script is imported to use in another Python script, the .py extension does not need to be added to the name of the file when using the import statement.
from config import weather_api_key


# Import the time module.
    #  import the time module to get the current date, rather than the datetime module that we used to convert the date.
import time

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# Import the requests library.
    # To request JSON data over the internet, we use the Requests Library in Python. 
import requests



# Deliverable 1: Retrieve Weather Data (25 points)

### 3. Create a new set of 2,000 random latitudes and longitudes.

In [2]:
# Create a set of random latitude and longitude combinations.
    # 1) Create arrays of latitudes and longitudes, we'll declare each array as a variable. 
    # 2) Use Numpty random uniform because its faster to generate the random latitudes and longtitudes.
            # To generate more than one floating-point decimal number between –90 and 90, we can add the size parameter when we use the NumPy module and set that equal to any whole number.
    # 3) Use zip() we'll pack the latitudes (lats) and longitudes (lngs) as pairs by zipping them (lat_lngs) with the zip() function.

lats = np.random.uniform(low=-90.00, high=90.000, size=1500)
lngs = np.random.uniform(-180.000, 180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs 

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_lngs)
coordinates[:3]

[(-12.484171919892049, 163.28541576741026),
 (-68.1776280445993, 99.5724880289115),
 (87.80119197887382, -108.29002564170132)]

### 4. Get the nearest city using the citipy module.

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)   

630

### 5. Perform an API call with the OpenWeatherMap.

In [5]:
# Starting URL for Weather Map API Call.
# Structure example = api.openweathermap.org/data/2.5/weather?q={city name},{state code},{country code}&appid={API key}
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

### 6. Retrieve the following information from the API call:

- Latitude and longitude
- Maximum temperature
- Percent humidity
- Percent cloudiness
- Wind speed
- Weather description- (for example, clouds, fog, light rain, clear sky)

In [6]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Create the for loop with the enumerate() method and reference the index and the city in the list.
    # Use the enumerate() method to get the index of the city for logging purposes and the city for creating an endpoint URL as an alternative 
    # way to iterate through the list of cities and retrieve both the index, and the city from the list.
for i, city in enumerate(cities):    
    
    # Group cities in sets of 50 for logging purposes.
        #In the conditional statement, we check if the remainder of the index divided by 50 is equal to 0 and if the index is greater than or equal to 50. 
        # If the statement is true, then the set_count and the record_count are incremented by 1.
    if (i % 50 == 0 and i >= 50):
        # % is modulo operation in python for example expression "5 mod 2" would evaluate to 1, because 5 divided by 2 has a quotient of 2 and a remainder of 1, while "9 mod 3" would evaluate to 0,
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city.
        # Inside the conditional statement, we create the URL endpoint for each city, as before. However, we are removing the blank spaces in the city name 
        # and concatenating the city name with, city.replace(" ","+"). This will find the corresponding weather data for the city instead of finding the weather data for the first part of the city name.
            # Compare the URLs for Cabo (Links to an external site.) and Cabo San Lucas
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
        # Add a print statement that tells us the record count and set count, and the city that is being processed.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON for the current city and retrieve data.
            # If there is no weather data for the city, i.e. , a <Response [404]> then there is no weather to retrieve and City not found. Skipping... is printed.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
            # If there is weather data for the city, we will retrieve the latitude, longitude, maximum temperature, humidity, cloudiness, wind speed, and date and assign those values to variables.
            # We could write a try-except block for each one of these parameters to handle the KeyError if the data wasn't found, but since these parameters are always present in the response this won't be necessary.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
            # We append the cities list with a dictionary for that city, where the key-value pairs are the values from our weather parameters.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Description": city_desc})

# If an error is experienced, skip the city.
    # Below the try block and after the except block, we add the closing print statement, which will let us know the data retrieval has been completed. 
    # Make sure that your except block is indented and in line with the try block, and that the print statements are flush with the margin.
    except:
        print("City not found. Skipping...")
        # Add the pass statement, which is a general purpose statement to handle all errors encountered and to allow the program to continue.
        # NOTE: Generally, it isn't good coding practice to add the pass statement to the except block. Ideally, we want to handle or catch each error as it happens and do something specific 
            # (e.g., add another try block or print out the error).
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kirakira
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | yellowknife
Processing Record 4 of Set 1 | roald
Processing Record 5 of Set 1 | derzhavinsk
Processing Record 6 of Set 1 | shimoda
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | quepos
Processing Record 9 of Set 1 | saint george
Processing Record 10 of Set 1 | verkhoyansk
Processing Record 11 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 12 of Set 1 | hobart
Processing Record 13 of Set 1 | bac lieu
Processing Record 14 of Set 1 | talnakh
Processing Record 15 of Set 1 | upernavik
Processing Record 16 of Set 1 | bluff
Processing Record 17 of Set 1 | lavrentiya
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | chapais
Processing Record 20 of Set 1 | beringovskiy
Processing Record 21 of Set 1 | tynda
Processing Record 22 of Set 1 | ushuaia
Processing Rec

In [7]:
len(city_data)

573

### 7. Add the data to a new DataFrame.

Before exporting your new DataFrame as a CSV file, take a moment to confirm that it looks similar to the image below:

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,Kirakira,-10.4544,161.9205,83.26,78,100,7.40,SB,2021-12-06 22:34:24,overcast clouds
1,Albany,42.6001,-73.9662,58.08,88,100,11.54,US,2021-12-06 22:30:48,light rain
2,Yellowknife,62.4560,-114.3525,-16.58,83,75,4.61,CA,2021-12-06 22:31:14,broken clouds
3,Roald,62.5833,6.1333,37.09,59,75,20.71,NO,2021-12-06 22:38:43,broken clouds
4,Derzhavinsk,51.1000,66.3167,20.82,97,100,5.28,KZ,2021-12-06 22:38:44,light snow
5,Shimoda,34.6667,138.9500,56.10,82,80,18.19,JP,2021-12-06 22:38:44,broken clouds
6,Rikitea,-23.1203,-134.9692,76.42,72,73,8.50,PF,2021-12-06 22:32:14,broken clouds
7,Quepos,9.4236,-84.1652,87.10,86,77,2.37,CR,2021-12-06 22:38:44,broken clouds
8,Saint George,37.1041,-113.5841,68.47,24,1,0.00,US,2021-12-06 22:32:12,clear sky
9,Verkhoyansk,67.5447,133.3850,-41.10,92,78,3.87,RU,2021-12-06 22:38:45,broken clouds


### 8. Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder.

In [9]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")